# Semantic SfM of single scene in Scannet

### 1. Download entire Scannet dataset

Find information on ScanNet website: http://www.scan-net.org/ScanNet/

After download the data, you should extract data from .sens using [SensReader/reader.py](https://github.com/ScanNet/ScanNet/tree/master/SensReader/python). All the extracted data should be stored in `output` folder under a scene folder, e.g., `scannet/scans/scene0000_00/output`.

```bash
python3 reader.py --filename /home/zchen256/semantic_SfM/data/scannet/scans/scene0000_00/scene0000_00.sens --output_path /home/zchen256/semantic_SfM/data/scannet/scans/scene0000_00/output --export_depth_images --export_color_images --export_poses --export_intrinsics
```

### 2. Extract Scannet data to prepare SSfM data

In [1]:
from scene_extractor import SceneExtractor

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
scene_dir = '../../data/scannet/scans/scene0000_00'
save_dir = '../../data/scene0000_00'

In [3]:
scene_extractor = SceneExtractor(scene_dir, save_dir)
scene_extractor.extract_photos()
scene_extractor.extract_depths()
scene_extractor.extract_segmentations()
scene_extractor.extract_reconstruction()
scene_extractor.extract_ground_truth()

Extracted 5578 segmentations


### 3. Select keyimages based on blur

In [3]:
from select_keyimages import select_scannet_keyimages

In [4]:
scene_dir = save_dir
select_scannet_keyimages(scene_dir, ratio=0.2, threshold=180, file_cluster_size=30, n_jobs=8)

Processing images:   0%|          | 0/5578 [00:00<?, ?it/s]

Processing images: 100%|██████████| 5578/5578 [00:29<00:00, 189.74it/s]


Number of keyimages_threshold:  2963
Number of keyimages_ratio:  1115
Number of keyimages:  2963
Number of selected keyimages:  195
Total images:  5578
Keyimages saved to:  ../../data/scene0000_00/associations/keyimages.yaml


### 4. Create projection associations

In [4]:
from ssfm.probabilistic_projection import *
import os
import yaml

In [5]:
pointcloud_projector = PointcloudProjection(depth_filtering_threshold=0.005)
pointcloud_projector.read_scannet_camera_parameters('../../data/scene0000_00')
pointcloud_projector.read_scannet_mesh('../../data/scene0000_00/reconstructions/mesh_vertices_color.npy')

In [6]:
# batch project
image_folder_path = '../../data/scene0000_00/photos'
save_folder_path = '../../data/scene0000_00/associations'

keyimages_path = os.path.join(save_folder_path, 'keyimages.yaml')

assert os.path.exists(keyimages_path), 'Keyimages not found'

with open(keyimages_path, 'r') as f:
    keyimages = yaml.safe_load(f)

# replace .npy with .jpg in keyimages, keyimages is a list of strings
image_list = [os.path.splitext(image)[0] + '.jpg' for image in keyimages]

In [7]:
#pointcloud_projector.parallel_batch_project(image_list, save_folder_path)
pointcloud_projector.parallel_batch_project_joblib(image_list, save_folder_path, num_workers=16, save_depth=False)

Processing frames: 100%|██████████| 195/195 [00:39<00:00,  4.88it/s]


In [8]:
# build keyimage associations
from ssfm.keyimage_associations_builder import *

In [9]:
smc_solver = KeyimageAssociationsBuilder('../../data/scene0000_00/associations', '../../data/scene0000_00/segmentations')
smc_solver.build_associations()

100%|██████████| 195/195 [00:00<00:00, 226.77it/s]


In [10]:
smc_solver.find_min_cover()

| Metric                                                       | Count      | Percentage           |
----------------------------------------------------------------------------------------------------
| Number of points not covered by any image                    | 17332      | 21.30                |
| Number of points covered by less than or equal to 1 image    | 30466      | 37.44                |
| Number of points covered by less than or equal to 3 images   | 44914      | 55.20                |
| Number of points covered by less than or equal to 5 images   | 54268      | 66.69                |


### 5. Estimate memory usage

In [12]:
from ssfm.memory_calculator import memory_calculator

In [13]:
# pointcloud file
npy_file = "../../data/scene0000_00/reconstructions/mesh_vertices_color.npy"
# image file sample
image_file = "../../data/scene0000_00/photos/0.jpg"
# number of images
num_images = len(image_list)
# number of segmentation ids for each point in the point cloud
num_segmentation_ids = 5

memory_calculator(npy_file, image_file, num_images, num_segmentation_ids)

+----------------------------------------+-----------------------+
|              Memory Type               |  Memory Required (GB) |
+----------------------------------------+-----------------------+
|      Segmentation for each image       |  0.002336740493774414 |
| Pixel2point association for each image |  0.004673480987548828 |
| Point2pixel association for each image | 0.0003031231462955475 |
|                                        |                       |
|      Segmentation for all images       |  0.45566439628601074  |
| Pixel2point association for all images |   0.9113287925720215  |
| Point2pixel association for all images |  0.05910901352763176  |
|          pc_segmentation_ids           | 0.0015156157314777374 |
|         pc_segmentation_probs          | 0.0015156157314777374 |
|          keyimage_association          |  0.01477725338190794  |
|                 Total                  |   1.4439106872305274  |
+----------------------------------------+--------------------

### 6. Run object registration

In [1]:
from ssfm.object_registration import *
from ssfm.post_processing import PostProcessing

In [2]:
pointcloud_path = '../../data/scene0000_00/reconstructions/mesh_vertices_color.npy'
segmentation_folder_path = '../../data/scene0000_00/segmentations'
image_folder_path = '../../data/scene0000_00/photos'
association_folder_path = '../../data/scene0000_00/associations'

keyimage_associations_file_name = 'associations_keyimage.npy'
keyimage_yaml_name = 'keyimages.yaml'

In [3]:
# Create object registration
obr = ObjectRegistration(pointcloud_path, segmentation_folder_path, association_folder_path, keyimage_associations_file_name=keyimage_associations_file_name, keyimage_yaml_name=keyimage_yaml_name)

# Run object registration
obr.object_registration(iou_threshold=0.5, save_semantics=True)


# save semantic point cloud
obr.save_semantic_pointcloud('../../data/scene0000_00/semantic_model.las')

Processing image 1/195: 0.npy
Processing image 2/195: 100.npy
Processing image 3/195: 1000.npy
Processing image 4/195: 1017.npy
Processing image 5/195: 1043.npy
Processing image 6/195: 1147.npy
Processing image 7/195: 1162.npy
Processing image 8/195: 1167.npy
Processing image 9/195: 1362.npy
Processing image 10/195: 138.npy
Processing image 11/195: 1385.npy
Processing image 12/195: 1400.npy
Processing image 13/195: 141.npy
Processing image 14/195: 1424.npy
Processing image 15/195: 1467.npy
Processing image 16/195: 1483.npy
Processing image 17/195: 1539.npy
Processing image 18/195: 1600.npy
Processing image 19/195: 1627.npy
Processing image 20/195: 1644.npy
Processing image 21/195: 1667.npy
Processing image 22/195: 167.npy
Processing image 23/195: 1696.npy
Processing image 24/195: 1700.npy
Processing image 25/195: 1749.npy
Processing image 26/195: 1753.npy
Processing image 27/195: 1791.npy
Processing image 28/195: 1800.npy
Processing image 29/195: 183.npy
Processing image 30/195: 1851.n

In [4]:
image_id = 194
semantics_folder_path = os.path.join(association_folder_path, 'semantics', 'semantics_{}.npy'.format(image_id))
save_las_path = os.path.join(association_folder_path, 'semantics', 'semantics_{}.las'.format(image_id))
add_semantics_to_pointcloud(pointcloud_path, semantics_folder_path, save_las_path) 

maximum of semantics:  1990
number of unique semantics:  99


In [5]:
semantic_pc_file_path = '../../data/scene0000_00/associations/semantics/semantics_{}.las'.format(image_id)
post_processing = PostProcessing(semantic_pc_file_path)
post_processing.shuffle_semantic_ids()
save_las_path = '../../data/scene0000_00/associations/semantics/semantics_{}_shuffled.las'.format(image_id)
post_processing.save_semantic_pointcloud(save_las_path)

Number of unique semantics:  99
